In [1]:
import railtracks as rt
from pydantic import BaseModel, Field
from copy import deepcopy
import textwrap

Make sure you have the environment set up correctly. For running this example successfully, you need the following API keys:
```
OPENAI_API_KEY
HF_TOKEN
```

For more information on how to set up the environment, please refer to the DOCS LINK HERE


In [2]:
openai_llm = rt.llm.OpenAILLM("gpt-4o")
hf_llm = rt.llm.HuggingFaceLLM(model_name="featherless-ai/mistralai/Mistral-7B-Instruct-v0.2")        # good for terminal LLM but doesn't support tool calling


class AnswerPrediction(BaseModel):
    quality_answer: bool = Field(
        ..., description="The quality of the answer that was provided by the model."
    )


system_c_o_t = rt.llm.SystemMessage(
    "You are extremley intellegnt reviewer of information that ponders the provided question and context to provide a simple one paragraph plan where you discuss you thinking process and how you would go about answering the question, without explictely answer the question."
)
system_answer = rt.llm.SystemMessage(
    "You are a helpful assistant that works tirelessly to answer a question using the provided context as a guide to answer the question."
)
system_answer_reviewer = rt.llm.SystemMessage(
    "You are a reviewer of the plan that was provided by the model and will determine if the plan is feasible and if it will accomplish the task that was asked of you. Be harsh and clear when the plan is not adequete."
)


COTNode = rt.agent_node("COT", system_message=system_c_o_t, llm_model=openai_llm)
AnswerNode = rt.agent_node(
    "Answerer", system_message=system_answer, llm_model=hf_llm
)
ReviewerNode = rt.agent_node(
    output_schema=AnswerPrediction,
    name="AnswerReviewer",
    system_message=system_answer_reviewer,
    llm_model=openai_llm,
)


async def COTLLM(message_history: rt.llm.MessageHistory, number_trails: int = 4):
    original_message_history = deepcopy(message_history)
    for _ in range(number_trails):
        cot_response = await rt.call(COTNode, user_input=message_history)

        message_history.append(rt.llm.AssistantMessage("My plan:" + cot_response.text))

        answer_response = await rt.call(ReviewerNode, user_input=message_history)
        if not answer_response.structured.quality_answer:
            continue
        else:
            break

    original_message_history.append(
        rt.llm.UserMessage("Contextual Plan: " + cot_response.text)
    )
    response = await rt.call(AnswerNode, user_input=original_message_history)

    return cot_response.text, response.text


ChainOfThought = rt.function_node(COTLLM)

In [3]:
message_history = rt.llm.MessageHistory([rt.llm.UserMessage("")])
with rt.Session() as runner:
    response = await rt.call(ChainOfThought, message_history=message_history)


Using existing session


[+3.330  s] RT          : INFO     - START CREATED COTLLM
[+3.333  s] RT          : INFO     - COTLLM CREATED COT
[+9.954  s] RT          : INFO     - COT DONE
[+9.957  s] RT          : INFO     - COTLLM CREATED AnswerReviewer
[+10.840 s] RT          : INFO     - AnswerReviewer DONE
[+10.840 s] RT          : INFO     - COTLLM CREATED Answerer
[+40.419 s] RT          : INFO     - Answerer DONE
[+40.422 s] RT          : INFO     - COTLLM DONE
[+40.424 s] RT.Session  : INFO     - Saving execution info to .railtracks\df73525c-caa9-4f5b-8672-9dab88a02e6e.json


In [4]:
print("COT plan: ")
for line in textwrap.wrap(response[0], width=200):
    print(line)
print("-------------------")
print("Answer: " + response[1])

COT plan: 
What is the role of social media in shaping public opinion?   To tackle the question of social media's role in shaping public opinion, I would first consider the historical context of media influences
on society and compare traditional media's influence to social media's, especially in terms of speed and reach. I would examine case studies where social media has played a pivotal role in shaping
public discourse, such as political campaigns, social movements, and public health information dissemination. Additionally, analyzing the algorithms behind social media platforms would reveal how
information is curated and presented to users, which could influence opinions. I would also explore the role of influencers and user-generated content in creating echo chambers or diversifying
perspectives. Finally, I would consider both the benefits and challenges, such as the rapid spread of misinformation versus the democratization of information access, in my evaluation. This multi-
facet